In [0]:
import numpy as np
RANDOM_STATE = 2019
np.random.seed(RANDOM_STATE) 

from google.colab import drive
drive.mount('/gdrive')

import pandas as pd
import time
import lightgbm as lgb

import seaborn as sns
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import KFold
from scipy import stats
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go

def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

import gc
import os

os.chdir("/gdrive/My Drive/Colab Notebooks/dacon")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
resampled_existing = pd.read_csv('data/comp_9th/resampled_existing_0819.csv',parse_dates=['date'])
resampled_w0 = pd.read_csv('data/comp_9th/resampled_w0_0819.csv',parse_dates=['date'])

In [0]:
def rename(df):
    df.rename(columns={ 'cycle_idx_week1_x': 'cycle_idx_week1' ,
                      'cycle_idx_week2_x': 'cycle_idx_week2',
                      'cycle_idx_week4_x': 'cycle_idx_week4'},
            inplace=True)
    df.rename(columns={ 'cycle_idx_week1_y': 'max_cidx_week1' ,
                    'cycle_idx_week2_y': 'max_cidx_week2',
                    'cycle_idx_week4_y': 'max_cidx_week4'},
          inplace=True)
    return df

resampled_w0 = rename(resampled_w0)
resampled_existing = rename(resampled_existing)


In [0]:
def check(colname):
    if ('h-day_cnt_' in colname) or ('pt_cnt_'  in colname) or ('inst_cnt_' in colname):
        if '13' in colname:
            return True
        return False
    else:
        return True

remained_col = [i for i in resampled_w0.columns.values if check(i)]
temp_col = [i for i in resampled_w0.columns.values if not check(i)]

In [0]:

resampled_existing_full = resampled_existing.copy()
resampled_existing_121 = resampled_existing[remained_col].copy()
resampled_w0_full = resampled_w0.copy()
resampled_w0_121 = resampled_w0[remained_col].copy()

del resampled_existing, resampled_w0

gc.collect()

421

In [0]:
!pip install scikit-optimize

     |████████████████████████████████| 81kB 9.1MB/s 


In [0]:
#!pip install scikit-optimize
#import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



In [0]:
def objective(values):
    params = {
              'max_depth': values[0],  # 0
              'task': 'train',
              'num_leaves': int((2**(values[0]) ) -1) , 
              'learning_rate': values[1],    # 1
              'feature_fraction': values[2], # 2
              'lambda_l1': values[3],       # 3
              'lambda_l2': values[4],         # 4
              'bagging_fraction': values[5], # 5
              'bagging_freq': values[6],       #  6
              'max_bin': values[7],           # 7
              'metric':'mae',
              'boosting_type': 'gbdt',
              'objective': 'regression_l1',
              'verbose': -1,
              'seed': RANDOM_STATE
              }

    print('\nNext set of params.....',params)
    print("with data:   "+values[8])
    if values[8] == 'full':
        resampled = resampled_w0_full.copy()
    elif values[8] =='121':
        resampled = resampled_w0_121.copy()

    real_train = resampled[resampled['date'] <= pd.to_datetime('2018-11-28') ]
    real_test = resampled[(resampled['Year'] == 2018) &
                          (resampled['Month'] == 11) &
                          (resampled['Day'] == 29) ]

    ##    
    original_target = real_train.y.values
    target, lambda_prophet = stats.boxcox(real_train['y'] + 1)
    len_train=target.shape[0]
    merged_df = pd.concat([real_train, real_test])
    merged_df.drop(['y','date'],axis=1,inplace=True)

    cate = ['store_id',  'Month', 'Week', 'Day', 'Dayofweek','Dayofyear', 
            'weekofyear','type_bus', 'high_region', 'full_region','cur_zero',
            'cycle_idx_week1', 'cycle_idx_week2', 'cycle_idx_week4']

    columns = merged_df.columns.values
    col_dict = {}
    for i,colname in enumerate(columns):
      col_dict[colname] = i
    cate_feat2num = [col_dict[colname] for colname in cate]

    # do the training
    num_folds = 3
    test_x = merged_df[len_train:].values
    all_x = merged_df[:len_train].values
    all_y = target # removing what we did earlier
    oof_preds = np.zeros([all_y.shape[0]])
    sub_preds = np.zeros([test_x.shape[0]])
    folds = KFold(n_splits=num_folds, shuffle=True, random_state=2019)
    ##
    num_boost_round = 3000
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(all_x)):
        start_time = time.time()
        train_x, train_y = all_x[train_idx], all_y[train_idx]
        valid_x, valid_y = all_x[valid_idx], all_y[valid_idx]
        lgb_train = lgb.Dataset(train_x,train_y)
        lgb_valid = lgb.Dataset(valid_x,valid_y)

        # train

        gbm = lgb.train(params, lgb_train, num_boost_round= num_boost_round, 
            valid_sets=[lgb_train, lgb_valid],
            early_stopping_rounds=100, verbose_eval=1500)
        
        oof_preds[valid_idx] = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
        sub_preds[:] += gbm.predict(test_x, num_iteration=gbm.best_iteration) / folds.n_splits
        valid_idx += 1

        print("--- %s seconds ---" % (time.time() - start_time))

    sub_preds = inverse_boxcox(sub_preds , lambda_prophet) - 1
    oof_preds = inverse_boxcox(oof_preds , lambda_prophet) - 1
    y_true = real_test['y'].values

    sub_preds = np.nan_to_num(sub_preds)
    mae_1129 = mean_absolute_error(y_true = y_true, y_pred= sub_preds )
    

    print(len(y_true))
    print("mae on 11/29  :"+ str(mae_1129) )
    
    gc.collect()
    
    return mae_1129



In [0]:
space  = [Integer(8, 12, name='max_depth'), # 0
          Real(0.05, 0.25,  name='learning_rate',prior='log-uniform',), # 1
          Real(0.7, 0.9, name='feature_fraction'), # 2
          Real(0.01, 0.2,  name='lambda_l1'), # 3
          Real(0.01, 0.2,  name='lambda_l2'), # 4
          Real(0.3, 0.9,  name='bagging_fraction'), # 5
          Integer(5, 100, name='bagging_freq'), # 6
          Integer(50, 255, name='max_bin'), #7
          Categorical(['full','121'], name='data') # 8
         ]

- https://scikit-optimize.github.io/#skopt.gp_minimize

In [0]:
# 0820
default_parameters = [10, 0.1, 0.8, 0.06, 0.1, 0.8 , 30, 128 ,'121' ]

n_calls = 20

res_gp = gp_minimize(objective, space, n_calls = n_calls,
                     random_state = RANDOM_STATE,
                     x0=default_parameters)



Next set of params..... {'max_depth': 10, 'task': 'train', 'num_leaves': 1023, 'learning_rate': 0.1, 'feature_fraction': 0.8, 'lambda_l1': 0.06, 'lambda_l2': 0.1, 'bagging_fraction': 0.8, 'bagging_freq': 30, 'max_bin': 128, 'metric': 'mae', 'boosting_type': 'gbdt', 'objective': 'regression_l1', 'verbose': -1, 'seed': 2019}
with data:   121
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 9.36669	valid_1's l1: 14.7675
[3000]	training's l1: 8.11785	valid_1's l1: 14.2498
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 8.11785	valid_1's l1: 14.2498
--- 220.92199087142944 seconds ---
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 9.39211	valid_1's l1: 14.6876
[3000]	training's l1: 8.17768	valid_1's l1: 14.168
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 8.17768	valid_1's l1: 14.168
--- 216.64696526527405 seconds ---
Training until validation scores don't improve for 100 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in log



{'max_depth': 8, 'task': 'train', 'num_leaves': 255, 'learning_rate': 0.11614746341176702, 'feature_fraction': 0.8091264808742366, 'lambda_l1': 0.10647214537333566, 'lambda_l2': 0.1083314750853381, 'bagging_fraction': 0.7346420453587098, 'bagging_freq': 46, 'max_bin': 232, 'metric': 'mae', 'boosting_type': 'gbdt', 'objective': 'regression_l1', 'verbose': -1, 'seed': 2019}
with data:   121
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 11.7118	valid_1's l1: 15.8951
[3000]	training's l1: 10.4937	valid_1's l1: 15.3617
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 10.4937	valid_1's l1: 15.3617
--- 164.26452469825745 seconds ---
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 11.9956	valid_1's l1: 15.9425
[3000]	training's l1: 10.6316	valid_1's l1: 15.3177
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 10.6316	valid_1's l1: 15.3177
--- 161.99536991119385 seconds ---
Trai

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in log



Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 12.4296	valid_1's l1: 16.1881
[3000]	training's l1: 11.3249	valid_1's l1: 15.7157
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 11.3249	valid_1's l1: 15.7157
--- 146.95588159561157 seconds ---
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 12.4683	valid_1's l1: 16.1567
[3000]	training's l1: 11.2283	valid_1's l1: 15.6146
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 11.2283	valid_1's l1: 15.6146
--- 148.65579462051392 seconds ---
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 12.6802	valid_1's l1: 16.2589
[3000]	training's l1: 11.4298	valid_1's l1: 15.7181
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 11.4298	valid_1's l1: 15.7181
--- 149.07768845558167 seconds ---
1967
mae on 11/29  :601256.5592810918

Next set of params..... {'max_depth': 11, 'task': 'trai

...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in log




Next set of params..... {'max_depth': 10, 'task': 'train', 'num_leaves': 1023, 'learning_rate': 0.15621336355425158, 'feature_fraction': 0.7990417978494787, 'lambda_l1': 0.13997764153058076, 'lambda_l2': 0.15706856215556883, 'bagging_fraction': 0.5510346435271056, 'bagging_freq': 74, 'max_bin': 148, 'metric': 'mae', 'boosting_type': 'gbdt', 'objective': 'regression_l1', 'verbose': -1, 'seed': 2019}
with data:   121
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 9.37325	valid_1's l1: 15.2715
[3000]	training's l1: 8.01934	valid_1's l1: 14.7959
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 8.01934	valid_1's l1: 14.7959
--- 180.7692358493805 seconds ---
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 9.57304	valid_1's l1: 15.3776
[3000]	training's l1: 8.14167	valid_1's l1: 14.8681
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 8.14167	valid_1's l1: 14.8681
--- 178.3231

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in log




Next set of params..... {'max_depth': 10, 'task': 'train', 'num_leaves': 1023, 'learning_rate': 0.1609988647127048, 'feature_fraction': 0.7208313114754651, 'lambda_l1': 0.11343401967624266, 'lambda_l2': 0.0337278499071973, 'bagging_fraction': 0.470438860535487, 'bagging_freq': 71, 'max_bin': 179, 'metric': 'mae', 'boosting_type': 'gbdt', 'objective': 'regression_l1', 'verbose': -1, 'seed': 2019}
with data:   121
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 9.7968	valid_1's l1: 15.6773
[3000]	training's l1: 8.27343	valid_1's l1: 15.1857
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 8.27343	valid_1's l1: 15.1857
--- 169.18260550498962 seconds ---
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 9.77053	valid_1's l1: 15.5925
[3000]	training's l1: 8.29032	valid_1's l1: 15.1052
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 8.29032	valid_1's l1: 15.1052
--- 167.7809388

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in log



In [0]:
report = pd.concat([pd.DataFrame(res_gp.x_iters, columns = 
                                 ["max_depth","learning rate","feature_fraction","lambda_l1",
                                  "lambda_l2","bagging_fraction","bagging_freq","max_bin","data"]),
            (pd.Series(res_gp.func_vals, name="mae"))], axis=1)
report.to_csv('./report_lgb_skopt_0820.csv', index = False)

In [0]:
report

,max_depth,learning rate,feature_fraction,lambda_l1,lambda_l2,bagging_fraction,bagging_freq,max_bin,data,mae
0,10,0.100000,0.800000,0.060000,0.100000,0.800000,30,128,121,557695.580924
1,8,0.116147,0.809126,0.106472,0.108331,0.734642,46,232,121,600922.540323
2,8,0.103753,0.709824,0.173703,0.033150,0.714968,88,122,full,601256.559281
3,11,0.108593,0.821212,0.019669,0.146586,0.605669,68,132,full,574533.498090
4,10,0.060485,0.703002,0.159119,0.097655,0.336710,59,197,121,612468.292379
5,9,0.065513,0.847109,0.177840,0.123538,0.608719,61,91,121,589026.256646
6,10,0.174109,0.705303,0.129104,0.036220,0.477329,96,151,121,582939.735980
7,8,0.078035,0.827387,0.084962,0.159953,0.724319,44,122,full,602596.283010
8,8,0.223667,0.796909,0.163224,0.065823,0.878930,39,225,full,607588.112227
9,9,0.061252,0.796260,0.126014,0.175829,0.471282,28,148,121,597177.726228


In [0]:
def objective(values):
    params = {
              'max_depth': values[0],  # 0
              'task': 'train',
              'num_leaves': int((2**(values[0]) ) -1) , 
              'learning_rate': values[1],    # 1
              'feature_fraction': values[2], # 2
              'lambda_l1': values[3],       # 3
              'lambda_l2': values[4],         # 4
              'bagging_fraction': values[5], # 5
              'bagging_freq': values[6],       #  6
              'max_bin': values[7],           # 7
              'metric':'mae',
              'boosting_type': 'gbdt',
              'objective': 'regression_l1',
              'verbose': -1,
              'seed': RANDOM_STATE
              }

    print('\nNext set of params.....',params)
    print("with data:   "+values[8])
    if values[8] == 'full':
        resampled = resampled_existing_full.copy()
    elif values[8] =='121':
        resampled = resampled_existing_121.copy()

    
    real_train = resampled[resampled['date'] <= pd.to_datetime('2018-11-28') ]
    real_test = resampled[(resampled['Year'] == 2018) &
                          (resampled['Month'] == 11) &
                          (resampled['Day'] == 29) ]

    ##    
    original_target = real_train.y.values
    target, lambda_prophet = stats.boxcox(real_train['y'] + 1)
    len_train=target.shape[0]
    merged_df = pd.concat([real_train, real_test])
    merged_df.drop(['y','date'],axis=1,inplace=True)

    cate = ['store_id',  'Month', 'Week', 'Day', 'Dayofweek','Dayofyear', 
            'weekofyear','type_bus', 'high_region', 'full_region','cur_zero',
            'cycle_idx_week1', 'cycle_idx_week2', 'cycle_idx_week4']

    columns = merged_df.columns.values
    col_dict = {}
    for i,colname in enumerate(columns):
        col_dict[colname] = i
    cate_feat2num = [col_dict[colname] for colname in cate]

    # do the training
    num_folds = 3
    test_x = merged_df[len_train:].values
    all_x = merged_df[:len_train].values
    all_y = target # removing what we did earlier
    oof_preds = np.zeros([all_y.shape[0]])
    sub_preds = np.zeros([test_x.shape[0]])
    folds = KFold(n_splits=num_folds, shuffle=True, random_state=2019)
    ##
    num_boost_round = 3000
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(all_x)):
        start_time = time.time()
        train_x, train_y = all_x[train_idx], all_y[train_idx]
        valid_x, valid_y = all_x[valid_idx], all_y[valid_idx]
        lgb_train = lgb.Dataset(train_x,train_y)
        lgb_valid = lgb.Dataset(valid_x,valid_y)

        # train

        gbm = lgb.train(params, lgb_train, num_boost_round= num_boost_round, 
            valid_sets=[lgb_train, lgb_valid],
            early_stopping_rounds=100, verbose_eval=1500)
        
        oof_preds[valid_idx] = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
        sub_preds[:] += gbm.predict(test_x, num_iteration=gbm.best_iteration) / folds.n_splits
        valid_idx += 1

        print("--- %s seconds ---" % (time.time() - start_time))

    sub_preds = inverse_boxcox(sub_preds , lambda_prophet) - 1
    oof_preds = inverse_boxcox(oof_preds , lambda_prophet) - 1
    y_true = real_test['y'].values

    sub_preds = np.nan_to_num(sub_preds)
    mae_1129 = mean_absolute_error(y_true = y_true, y_pred= sub_preds )
    

    print(len(y_true))
    print("mae on 11/29  :"+ str(mae_1129) )
    
    gc.collect()
    
    return mae_1129



In [0]:
space  = [Integer(8, 12, name='max_depth'), # 0
          Real(0.05, 0.25,  name='learning_rate',prior='log-uniform',), # 1
          Real(0.7, 0.9, name='feature_fraction'), # 2
          Real(0.01, 0.2,  name='lambda_l1'), # 3
          Real(0.01, 0.2,  name='lambda_l2'), # 4
          Real(0.3, 0.9,  name='bagging_fraction'), # 5
          Integer(5, 100, name='bagging_freq'), # 6
          Integer(50, 255, name='max_bin'), #7
          Categorical(['full','121'], name='data') # 8
         ]

In [0]:
default_parameters = [10, 0.1, 0.8, 0.06, 0.1, 0.8 , 30, 128 ,'121' ]

res_gp = gp_minimize(objective, space, n_calls = 30,
                     random_state = RANDOM_STATE,
                     x0=default_parameters)

report = pd.concat([pd.DataFrame(res_gp.x_iters, columns = 
                                 ["max_depth","learning rate","feature_fraction","lambda_l1",
                                  "lambda_l2","bagging_fraction","bagging_freq","max_bin","data"]),
            (pd.Series(res_gp.func_vals, name="mae"))], axis=1)
report.to_csv('./report_lgb_skopt_0820_existing.csv', index = False)



Next set of params..... {'max_depth': 10, 'task': 'train', 'num_leaves': 1023, 'learning_rate': 0.1, 'feature_fraction': 0.8, 'lambda_l1': 0.06, 'lambda_l2': 0.1, 'bagging_fraction': 0.8, 'bagging_freq': 30, 'max_bin': 128, 'metric': 'mae', 'boosting_type': 'gbdt', 'objective': 'regression_l1', 'verbose': -1, 'seed': 2019}
with data:   121
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 0.812079	valid_1's l1: 1.32604
[3000]	training's l1: 0.701364	valid_1's l1: 1.28319
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 0.701364	valid_1's l1: 1.28319
--- 192.3959081172943 seconds ---
Training until validation scores don't improve for 100 rounds.
[1500]	training's l1: 0.791352	valid_1's l1: 1.30737
[3000]	training's l1: 0.690739	valid_1's l1: 1.26983
Did not meet early stopping. Best iteration is:
[3000]	training's l1: 0.690739	valid_1's l1: 1.26983
--- 192.53298091888428 seconds ---
Training until validation scores don't improve f

In [0]:
report.sort_values('mae',ascending=True).head(30)

,max_depth,learning rate,feature_fraction,lambda_l1,lambda_l2,bagging_fraction,bagging_freq,max_bin,data,mae
24,12,0.086645,0.809385,0.074108,0.086937,0.795203,83,125,121,551332.893396
20,10,0.117011,0.762130,0.048061,0.117977,0.822445,36,128,121,551690.056420
16,10,0.150782,0.710703,0.021155,0.144017,0.829142,37,128,121,554685.862716
14,10,0.116255,0.752845,0.074187,0.155478,0.736164,56,128,121,555315.106917
12,11,0.096715,0.773241,0.072301,0.088829,0.774110,35,124,121,556869.919455
22,10,0.235698,0.707686,0.031304,0.172524,0.793550,37,125,121,559044.915488
0,10,0.100000,0.800000,0.060000,0.100000,0.800000,30,128,121,560240.933205
26,11,0.098576,0.765801,0.115313,0.083674,0.796310,73,125,121,560468.487637
11,10,0.097708,0.777576,0.067233,0.085458,0.752110,40,128,121,560733.594329
21,10,0.118010,0.762975,0.047523,0.117712,0.833360,35,128,121,563630.044953


- References:
    - https://www.kaggle.com/a31314431/bayesian-optimization-lightgbm
    - https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
    - https://zzsza.github.io/data/2018/09/03/how-to-win-a-data-science-competition-week4/

